# Soccer Prediction Model

#### Importing needed libs and setting correct system paths

In [1]:
import sys
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.cm as cm
from sklearn.metrics import f1_score 
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'games_played', 'round', 'current_formation']
all_models = ['knn', 'log', 'svc', 'gnb', 'randomForest']
leagues = model_libs.get_leagues_country_codes() # = { "epl": 'ENG' }
teams = form_data.get_teams()
league_rounds = model_libs.get_leagues_rounds()

""" Change depeding on which model you want to run"""
target = "points" # converted_goals or points

""" this variable 'testing' should be False if using CSV's and not pulling from the database. 
    - Testers won't have database.
    - Otherwise will build all data from scratch... takes a long time """
testing = False

INITIALIZED...


#### Pulling in the data either from the Database or the CSV (CSV for testers). Not really needed for testers...

In [2]:
data_csv = 'raw_data.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)
print('Data Loaded...')
print("Dataset size :: {}".format(raw_data.shape))
display(raw_data.head())

Data Loaded...
Dataset size :: (634, 47)


,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,current_record,opp_record,goals_for,opp_goals_for,goals_against,opp_goals_against,rpi,goals,points,current_team_possession,current_team_yellow_cards,current_team_goal_attempts,current_team_dangerous_attacks,current_team_sec_half_goals,current_team_saves,current_team_corner_kicks,current_team_ball_safe,current_team_first_half_goals,current_team_shots_on_target,current_team_attacks,current_team_goal_attempts_allowed,current_team_goal_kicks,current_team_shots_total,opp_team_possession,opp_team_yellow_cards,opp_team_goal_attempts,opp_team_dangerous_attacks,opp_team_sec_half_goals,opp_team_saves,opp_team_corner_kicks,opp_team_ball_safe,opp_team_first_half_goals,opp_team_shots_on_target,opp_team_attacks,opp_team_goal_attempts_allowed,opp_team_goal_kicks,opp_team_shots_total
0,249.0,21,FC Dallas,33.0,DC United,2016-03-26 21:30:00,4,3.0,0,4-2-3-1,0.666667,0.357143,4,2,5,5,0.630952,3.0,3.0,151.0,6.0,35.0,182.0,3,5.0,20.0,256.0,1,20.0,310.0,16.0,17.0,35.0,138.0,8.0,22.0,150.0,1,4.0,16.0,283.0,1,11.0,293.0,20.0,23.0,22.0
1,255.0,21,FC Dallas,39.0,Columbus Crew,2016-04-03 01:00:00,5,3.0,1,4-2-3-1,0.714286,0.166667,5,2,5,4,0.465919,1.0,1.0,152.0,7.0,32.0,169.0,3,5.0,14.0,284.0,2,17.0,336.0,19.0,16.0,32.0,171.0,3.0,39.0,194.0,2,12.0,22.0,248.0,0,18.0,289.0,22.0,15.0,37.0
2,265.0,21,FC Dallas,32.0,San Jose Earthquakes,2016-04-10 01:00:00,6,3.0,1,4-4-2,0.833333,0.500000,6,4,1,5,0.547723,2.0,1.0,148.0,6.0,45.0,168.0,4,9.0,10.0,286.0,2,19.0,339.0,19.0,17.0,45.0,145.0,7.0,19.0,151.0,2,7.0,11.0,308.0,2,11.0,309.0,28.0,31.0,19.0
3,272.0,21,FC Dallas,30.0,Portland Timbers,2016-04-14 02:30:00,7,3.0,0,4-4-2,0.687500,0.357143,6,4,3,7,0.497669,3.0,3.0,154.0,8.0,41.0,145.0,3,8.0,15.0,270.0,3,17.0,310.0,21.0,16.0,41.0,143.0,8.0,24.0,164.0,4,9.0,27.0,273.0,0,11.0,300.0,23.0,19.0,24.0
4,282.0,21,FC Dallas,27.0,Sporting Kansas City,2016-04-17 23:00:00,7,3.0,1,4-4-2,0.687500,0.625000,6,4,3,2,0.497669,2.0,3.0,154.0,8.0,41.0,145.0,3,8.0,15.0,270.0,3,17.0,310.0,21.0,16.0,41.0,143.0,3.0,22.0,152.0,3,16.0,10.0,296.0,1,9.0,317.0,35.0,32.0,22.0


## FORMATTING

#### Implementing the quartiles rankings and outputs to ranked_data.csv

In [3]:
# Helper Function - Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

""" Need to do some formatting of the Data before we run the models"""
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

if testing:
    ranked_data = form_data.get_rankings(leagues, teams, league_rounds, raw_data, False)
    ranked_data.to_csv('ranked_data.csv')          
else:
    ranked_data = pd.read_csv('ranked_data.csv')
    ranked_data = ranked_data.drop(ranked_data.columns[[0]], axis=1)

print('Ranked Data Loaded...')

Ranked Data Loaded...


### BUILDING CLASSIFICATION MODEL

- If using 'converted_goals' as target converts the goals to binary ( 0-1 or 2+ goals)
- Also the area where we transform the data as desired whether using ratios or the numbers themselves
- Then we remove the unwanted data from our DF
- Build a simple model to confirm it's working and then check the accuracy of that model

In [4]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    #display(new_data.head())
    (y, X) = model_libs._extract_target(new_data, target)
    models = form_model.train_models(round_number, X, y, models)
    return models


""" Formatting data to convert goals scored to the correct category"""
formatted_data = ranked_data.copy()

if target == "converted_goals":
    # Not using points as a target for this version, using goals
    formatted_data['converted_goals'] = formatted_data.apply(lambda row: model_libs.set_group(row['goals']), axis=1)
    formatted_data = formatted_data.drop(['points', 'goals'], 1)
else: 
    formatted_data = formatted_data.drop(['goals'], 1)

    """ This is where you manipulate the features as desired """
""" //////////////////////////////////////////////////////////////////////////////////////////////////// """
""" Using diff_squared methods for features """
""" //////////////////////////////////////////////////////////////////////////////////////////////////// """
formatted_data["diff_goals_for"] = formatted_data.apply(lambda row: model_libs.diff_square(row["goals_for"], row["opp_goals_for"]), axis=1)
formatted_data["diff_goals_allowed"] = formatted_data.apply(lambda row: model_libs.diff_square(row["goals_against"], row["opp_goals_against"]), axis=1)
formatted_data["diff_attacks"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_attacks"], row["opp_team_attacks"]), axis=1)   
formatted_data["diff_dangerous_attacks"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_dangerous_attacks"], row["opp_team_dangerous_attacks"]), axis=1)   
formatted_data["diff_goal_attempts"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_goal_attempts"], row["opp_team_goal_attempts"]), axis=1)
formatted_data["diff_ball_safe"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_ball_safe"], row["opp_team_ball_safe"]), axis=1)                                                                                                                                                           
formatted_data["diff_possession"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_possession"], row["opp_team_possession"]), axis=1)
#formatted_data["diff_corner_kicks"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_corner_kicks"], row["opp_team_corner_kicks"]), axis=1)                                                                                                                                                           
#formatted_data["diff_goal_kicks"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_goal_kicks"], row["opp_team_goal_kicks"]), axis=1)
#formatted_data["diff_saves"] = formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_saves"], row["opp_team_saves"]), axis=1)

columns_to_drop = ['current_record', 'opp_record', 'goals_for', 'opp_goals_for', 'goals_against', 'opp_goals_against', 'rpi']

_, stats = form_data.get_columns()

formatted_data = formatted_data.drop(ignore_cols + columns_to_drop + stats, 1)

#### Running ALL Features 
if target == "converted_goals":
    models_test_1 = run_features(formatted_data, [], 'converted_goals', ["knn"])
    (formatted_y, formatted_X) = model_libs._extract_target(formatted_data, 'converted_goals')
else:
    models_test_1 = run_features(formatted_data, [], 'points', ["knn"])
    (formatted_y, formatted_X) = model_libs._extract_target(formatted_data, 'points')
    
print(formatted_X.columns)

# Simple Function to check the accuracy of the models.  Not the Final function
def check_accuracy(model, data_X, y):
    actual_y = pd.DataFrame(y.values, columns=['actual'])
    predictions = pd.concat([pd.DataFrame(model.predict(data_X), columns=['predictions']), actual_y], axis=1)
    predictions['accuracy'] = predictions.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
    accuracy = np.divide(predictions['accuracy'].sum(), float(len(predictions['accuracy'])))
    print(accuracy)

for m in models_test_1:
    check_accuracy(m, formatted_X, formatted_y)

-----------------------------------
Training K Neighbors Classifier Model
KNN Score on Training Set :: 0.6370808678500987
KNN Score on Test Set:: 0.29133858267716534
Finished K-Means Modeling
Index(['is_home', 'offensive_ranking', 'opp_defensive_ranking', 'rpi_ranking',
       'opp_rpi_ranking', 'diff_goals_for', 'diff_goals_allowed',
       'diff_attacks', 'diff_dangerous_attacks', 'diff_goal_attempts',
       'diff_ball_safe', 'diff_possession'],
      dtype='object')
0.567823343849


### Run, tune, and save all models 

In [5]:
model_results = []
for m in all_models:
    r = form_model.build_tuned_model(formatted_X, formatted_y, m)
    model_results.append(r)
    print('Accuracy :: ')
    check_accuracy(r[0], formatted_X, formatted_y)

-----------------------------------
Training K-Means Model
KNN Score :: 0.6613861386138614 for Training
KNN Score :: 0.3023255813953488 for Testing
KNN Score :: 0.6390532544378699 for Training
KNN Score :: 0.28346456692913385 for Testing
KNN Score :: 0.6673228346456693 for Training
KNN Score :: 0.3333333333333333 for Testing
KNN Score :: 0.6200787401574803 for Training
KNN Score :: 0.3333333333333333 for Testing
KNN Score :: 0.6259842519685039 for Training
KNN Score :: 0.30158730158730157 for Testing
Finished K-Means Modeling
Accuracy :: 
0.561514195584
Training LOG REG Model
Score on Training Set :: 0.4930966469428008
Score on Test Set :: 0.47244094488188976
Finished LOG REG Modeling
Accuracy :: 
0.488958990536
Training and Tuning SVC Model
[ 0.34251969  0.33992095]
Accuracy: 0.34 (+/- 0.00)
Finished SVC Modeling
Accuracy :: 
0.86119873817
-----------------------------------
Training Gaussian NB Model
[ 0.42718447  0.33333333  0.42574257  0.38613861  0.39      ]
Accuracy: 0.39 (+/- 0.

### Models have been saved so let's import them into one object.

In [6]:
prediction_models = form_model.load_models(all_models)

Success :: Loaded - knn
Success :: Loaded - log
Success :: Loaded - svc
Success :: Loaded - gnb
Success :: Loaded - randomForest


### Pulling the upcoming matches for the week and then ranking the quartiles like we did on the previous data

In [7]:
print('Upcoming matches')
#upcoming_matches, match_details = predict_matches.get_upcoming_matches()
#upcoming_matches.to_csv('upcoming_matches.csv')
upcoming_matches = pd.read_csv('upcoming_matches.csv')
upcoming_matches = upcoming_matches.drop(upcoming_matches.columns[[0]], axis=1)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
upcoming_data = predict_matches.predictions(upcoming_matches)

if testing:
    upcoming_ranked_data = form_data.get_rankings(leagues, teams, league_rounds, upcoming_data, True)
    upcoming_ranked_data.to_csv('upcoming_ranked_data.csv')               
else:
    upcoming_ranked_data = pd.read_csv('upcoming_ranked_data.csv')
    upcoming_ranked_data = upcoming_ranked_data.drop(upcoming_ranked_data.columns[[0]], axis=1)

print('Loaded Upcoming Data...')
#display(upcoming_ranked_data.head())

Upcoming matches
Loaded Upcoming Data...


### Formatting the features as need and as we did in the previous features of the build

In [8]:
""" Formatting data to convert goals scored to the correct category"""
upcoming_formatted_data = upcoming_ranked_data.copy()

""" //////////////////////////////////////////////////////////////////////////////////////////////////// """
""" Using diff_squared methods for features """
""" //////////////////////////////////////////////////////////////////////////////////////////////////// """
upcoming_formatted_data["diff_goals_for"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["goals_for"], row["opp_goals_for"]), axis=1)
upcoming_formatted_data["diff_goals_allowed"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["goals_against"], row["opp_goals_against"]), axis=1)
upcoming_formatted_data["diff_attacks"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_attacks"], row["opp_team_attacks"]), axis=1)   
upcoming_formatted_data["diff_dangerous_attacks"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_dangerous_attacks"], row["opp_team_dangerous_attacks"]), axis=1)   
upcoming_formatted_data["diff_goal_attempts"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_goal_attempts"], row["opp_team_goal_attempts"]), axis=1)
upcoming_formatted_data["diff_ball_safe"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_ball_safe"], row["opp_team_ball_safe"]), axis=1)                                                                                                                                                           
upcoming_formatted_data["diff_possession"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_possession"], row["opp_team_possession"]), axis=1)
#upcoming_formatted_data["diff_corner_kicks"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_corner_kicks"], row["opp_team_corner_kicks"]), axis=1)                                                                                                                                                           
#upcoming_formatted_data["diff_goal_kicks"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_goal_kicks"], row["opp_team_goal_kicks"]), axis=1)
#upcoming_formatted_data["diff_saves"] = upcoming_formatted_data.apply(lambda row: model_libs.diff_square(row["current_team_saves"], row["opp_team_saves"]), axis=1)

columns_to_drop = ['current_record', 'opp_record', 'goals_for', 'opp_goals_for', 'goals_against', 'opp_goals_against', 'rpi']
_, stats = form_data.get_columns()

upcoming_formatted_data = upcoming_formatted_data.drop(ignore_cols + columns_to_drop + stats + ['points', 'goals'], 1)
print(upcoming_formatted_data.columns)

Index(['is_home', 'offensive_ranking', 'opp_defensive_ranking', 'rpi_ranking',
       'opp_rpi_ranking', 'diff_goals_for', 'diff_goals_allowed',
       'diff_attacks', 'diff_dangerous_attacks', 'diff_goal_attempts',
       'diff_ball_safe', 'diff_possession'],
      dtype='object')


### Using the prediction_models object to predict the upcoming matches

In [9]:
""" Models we'll use to predict on upcoming matches """

# This is all the X values
upcoming_formatted_data

rf_preds = prediction_models[4].predict(upcoming_formatted_data)
print(rf_preds)

knn_preds = prediction_models[0].predict(upcoming_formatted_data)
print(knn_preds)

svc_preds = prediction_models[2].predict(upcoming_formatted_data)
print(svc_preds)

log_preds = prediction_models[1].predict(upcoming_formatted_data)
print(log_preds)

log_prob = prediction_models[1].predict_proba(upcoming_formatted_data)
probs = pd.DataFrame(log_prob)
#probs.to_csv('probs.csv')

gnb_preds = prediction_models[3].predict(upcoming_formatted_data)
print(gnb_preds)


[ 1.  0.  1.  3.  0.  3.  0.  3.  0.  3.  3.  3.  3.  3.  0.  0.  1.  3.
  0.  1.  3.  0.  3.  0.  0.  3.  3.  0.  0.  0.  0.  0.  3.  0.  3.  3.
  0.  3.  0.  3.  3.  0.  0.  0.  0.  3.  1.  3.  3.  3.  0.  0.  3.  1.
  3.  0.  0.  3.  3.  1.  0.  3.  0.  3.  1.  1.  1.  3.  0.  1.  0.  3.
  3.  0.  0.  0.  1.  1.  3.  1.  0.  1.  0.  0.  3.  0.  1.  3.  1.  0.
  3.  0.  3.  1.  3.  3.  0.  3.]
[ 1.  0.  0.  1.  0.  0.  0.  1.  1.  1.  0.  3.  3.  3.  0.  0.  1.  0.
  0.  1.  1.  3.  0.  1.  0.  0.  0.  3.  1.  0.  0.  0.  0.  1.  1.  3.
  3.  0.  1.  1.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  3.  0.  0.  0.
  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  3.  0.  1.  0.
  1.  1.  0.  0.  1.  0.  3.  0.  1.  1.  0.  0.  1.  1.  0.  3.  1.  3.
  0.  1.  0.  0.  0.  1.  0.  0.]
[ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.

## Verifying the results

 - Takes the upcoming matches and adds the previous predictions to that DF.  
 - Also adds a random series to that DF as a baseline
 - Since the upcoming_matches DF is ordered by Team Id, we reorder it so that the Current Team row and the Opponent Team Row are next to each other for easier visualisation.
 - Adds the actual_results dataframe so we can compare the results side by side
 - Spits the results out into a CSV for visualisation

In [10]:
columns = ['team_name', 'opp_name', 'scheduled', 'is_home']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]

if target == 'converted_goals':
    random_preds = pd.Series(np.random.randint(2, size=len(upcoming_matches.index)), upcoming_matches.index)
else:
    random_preds = pd.Series(np.random.randint(3, size=len(upcoming_matches.index)), upcoming_matches.index)
    random_preds[random_preds == 2] = 3

# Add predictions to the end of that DF
results = pd.DataFrame({'KNN': knn_preds, 'RandomForest': rf_preds, 'SVC': svc_preds, 'GNB': gnb_preds, 'log': log_preds, 'random': random_preds})
upcoming_matches = pd.concat([upcoming_matches, results], axis = 1)
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'is_home', 'KNN', 'RandomForest', 'SVC', 'GNB', 'log', 'random']
reordered_matches = reordered_matches[columns]
actual_results = pd.read_csv('actual_results.csv')
actual_results=actual_results.rename(columns = {'Unnamed: 0':'idx'})
indexed_results = actual_results.set_index('idx')
reordered_matches = pd.concat([reordered_matches, indexed_results], axis=1)
reordered_matches = reordered_matches.reset_index(drop=True)
display(reordered_matches.head(5))
reordered_matches.to_csv('predictions.csv')
print('Prediction CSV saved')

,scheduled,team_name,opp_name,is_home,KNN,RandomForest,SVC,GNB,log,random,actual,actual_converted_goals
0,2016-09-25 01:30:00,FC Dallas,Real Salt Lake,0.0,1.0,1.0,3.0,0.0,0.0,3.0,1,0
1,2016-09-25 01:30:00,Real Salt Lake,FC Dallas,1.0,0.0,1.0,3.0,3.0,3.0,3.0,1,0
2,2016-09-24 23:00:00,Colorado Rapids,Vancouver Whitecaps FC,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1,1
3,2016-09-24 23:00:00,Vancouver Whitecaps FC,Colorado Rapids,1.0,1.0,3.0,3.0,1.0,3.0,0.0,1,1
4,2016-09-24 00:00:00,New York City FC,Chicago Fire,1.0,1.0,3.0,3.0,3.0,3.0,3.0,3,1


Prediction CSV saved


## F1 Score

In [ ]:
column_model = "GNB"  #'KNN', 'RandomForest', 'SVC', 'GNB', 'log', 'random'

y_pred = reordered_matches[column_model]

if target == 'converted_goals':
    y_true = reordered_matches["actual_converted_goals"]
    display(reordered_matches[['actual_converted_goals', column_model]].head(10))
else:
    y_true = reordered_matches["actual"]
    display(reordered_matches[['actual', column_model]].head(10))


print(f1_score(y_true, y_pred, average='weighted'))

#### For converted_goals just checks to see the accuracy of the binary classifier.  For the points it gives more detail on the result.  Just need to change the column model to see which classifier you want to see the results for

In [ ]:
if target == 'converted_goals':
    actual = "actual_converted_goals"
    reordered_matches = pd.read_csv('predictions_converted_goals.csv')
    reordered_matches = reordered_matches.drop(reordered_matches.columns[[0]], axis=1)
    reordered_matches['accuracy'] = reordered_matches.apply(lambda r: model_libs.predictions_diff(r[actual], r[column_model]), axis=1)
    accuracy = np.divide(reordered_matches['accuracy'].sum(), float(len(reordered_matches['accuracy'])))
    #display(reordered_matches[[column_model, actual]])
    print(accuracy)
else:
    actual = "actual"
    results_data = reordered_matches.copy()
    home_actual_win = 0
    home_predicted_win = 0
    predicted_draws = 0
    actual_draws = 0
    valid_matches = 0
    total_matches = 0
    invalid_predictions = []
    correct_predictions = 0

    for r, rows in results_data.iterrows():
        if r % 2 == 0:
            total_matches += 1
            predictions = results_data.loc[r:r+1, ["is_home", column_model, actual]]
            home_team = predictions[predictions["is_home"] == 1]
            #print(predictions)
            if home_team.iloc[0]["actual"] == 3:
                home_actual_win += 1
            elif home_team.iloc[0]["actual"] == 1:
                actual_draws += 1

            # Check if it's a valid prediction (0-3, 3-0, 1-1)
            is_valid = False
            if ((predictions.iloc[0][column_model] == 1) & (predictions.iloc[1][column_model] == 1)) or ((predictions.iloc[0][column_model] == 3) & (predictions.iloc[1][column_model] == 0)) or ((predictions.iloc[0][column_model] == 0) & (predictions.iloc[1][column_model] == 3)):
                is_valid = True
                valid_matches += 1

                if home_team.iloc[0][column_model] == 3:
                    home_predicted_win += 1

                    if home_team.iloc[0]["actual"] == 3:
                        correct_predictions += 1

                if (predictions.iloc[0][column_model] == 1) & (predictions.iloc[1][column_model] == 1):
                    predicted_draws += 1

                    if home_team.iloc[0]["actual"] == 1:
                        correct_predictions += 1
            else:
               invalid_predictions.append(predictions) 

    print(column_model)        
    print('Total Matches :: {}'.format(total_matches))
    print('Valid Predicted Matches :: {}'.format(valid_matches))
    print('Actual Home Team Wins :: {}'.format(home_actual_win))
    print('Home Predicted Wins :: {}'.format(home_predicted_win))
    print('Actual Draws :: {}'.format(actual_draws))
    print('Predicted Draws :: {}'.format(predicted_draws))
    print('Correct Predictions :: {}'.format(correct_predictions))
    
    results_data['accuracy'] = results_data.apply(lambda r: model_libs.predictions_diff(r[column_model], r[actual]), axis=1)
    accuracy = np.divide(results_data['accuracy'].sum(), float(len(results_data['accuracy'])))
    print('Individual Accuracy :: {}'.format(accuracy))

    #print(invalid_predictions)
    

#### Removes half of the matches and verifies the accuracy of those predictions

In [ ]:
if target == "points":
    abbreviated_matches = reordered_matches.iloc[1::2, :]
    #abbreviated_matches = reordered_matches.iloc[::2, :]
    abbreviated_matches['accuracy'] = abbreviated_matches.apply(lambda r: model_libs.predictions_diff(r[column_model], r["random"]), axis=1)
    accuracy = np.divide(abbreviated_matches['accuracy'].sum(), float(len(abbreviated_matches['accuracy'])))
    print('Accuracy :: {}'.format(accuracy))

In [ ]:
reordered_matches['accuracy'] = reordered_matches.apply(lambda r: model_libs.predictions_diff(r['log'], r['actual']), axis=1)
home_score = pd.crosstab(reordered_matches['is_home'], reordered_matches['actual'])
home_score['Total'] = home_score.sum(axis=1)
home_score.loc['Total']= home_score.sum()
display(home_score)